# d_zonalst&cluster_data_combine_visualize is designed to combine Cluster and building parameters for easier visualization of subsequent indicators.

This code is intended to extract X, Y, and Cluster data separately, and then combine it with canopy and building parameters to generate the final_cities_cluster_with_index.shp file.

Here is the specific process breakdown:

#### 10 Data Preprocessing
> 10_01 Extract X, Y, 'Place', 'City Name', 'Place ID','Sum ID' and 'Cluster' data separately to obtain an original CSV file.
> 10_02 csv file into shapefile
> 10_03 Convert the shapefile to WGS 1984.

#### 11 Indicator Visualization
> 11_01 This code aims to normalize the 'MEAN', 'FAR', and 'biodensity' indicators.
> 11_01 shapefile into csv
> 11_02 Plot the corresponding indicator images.

# 10_01 Extract X, Y, 'Place', 'City Name', 'Place ID','Sum ID' and 'Cluster' data separately to obtain an original CSV file.

In [1]:
import pandas as pd
import os
import numpy as np
K_range = [11]
for K in K_range:
    cluster_origin_file=rf'D:\file\d_som\223_cities_combined_moving_ave_cluster_sort.csv'
    output_file_path=rf'D:\file\d_som'
    # 创建输出文件所在的目录
    os.makedirs(output_file_path, exist_ok=True)

    data = pd.read_csv(cluster_origin_file)
    data_xy = data.iloc[:, 0:2]  # Assuming that the first two columns are not features
    # 将结果保存为CSV文件，并加入前两列数据
    df = pd.DataFrame(data_xy)
    # 确保这些列存在
    for col in ['Place', 'City Name', 'Place ID','Sum ID', 'Cluster']:
        if col in data.columns:
            df[col] = data[col]

    output_file =  os.path.join(output_file_path,rf'223_cities_combined_moving_ave_cluster_sort_xy.csv')
    df.to_csv(output_file, index=False)

# 10_02 csv file into shapefile

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 定义 K 值

# 定义文件路径
csv_file = r"D:\file\d_som/223_cities_combined_moving_ave_cluster_sort_xy.csv"
shp_file = r"D:\file\d_som/223_cities_combined_moving_ave_cluster_sort_xy.shp"

# 读取 CSV 文件
df = pd.read_csv(csv_file)

# 创建 GeoDataFrame
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# 定义坐标系
# 使用 PROJ 字符串定义 Sinusoidal 投影
sinusoidal_proj = "+proj=sinu +R=6371007.181 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
gdf.set_crs(sinusoidal_proj, inplace=True)  # 设置输入坐标系为 Sinusoidal

# 将坐标系转换为 WGS 1984
gdf = gdf.to_crs(epsg=4326)  # WGS 1984 的 EPSG 代码是 4326

# 保存为 Shapefile
gdf.to_file(shp_file)

print("Shapefile 生成完成：", shp_file)


Shapefile 生成完成： D:\file\d_som/223_cities_combined_moving_ave_cluster_sort_xy.shp


# 10_03 Convert the shapefile to WGS 1984.

In [3]:
import geopandas as gpd

# 设置输入和输出文件路径
input_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp"
output_shapefile = r"D:\file\e_index\223_cities_cluster_with_index.shp"
projection_file = r"D:\file\d_som\223_cities_combined_moving_ave_cluster_sort_xy.shp"

# 读取输入 Shapefile
gdf = gpd.read_file(input_shapefile)

# 读取投影 Shapefile
proj_gdf = gpd.read_file(projection_file)

# 1. 将 gdf 和 proj_gdf 先转换为 UTM 坐标系（以第一个点的 UTM 区域为基准）
utm_crs = gdf.estimate_utm_crs()
gdf = gdf.to_crs(utm_crs)
proj_gdf = proj_gdf.to_crs(utm_crs)

# 3. 进行空间连接
gdf = gdf.sjoin(proj_gdf[['geometry', 'Cluster', 'Place', 'Place ID', 'Sum ID', 'City Name']], how='left', predicate='intersects')

# 将 Cluster 字段重命名为 K_11
gdf.rename(columns={'Cluster': 'Cluster'}, inplace=True)

# 4. 转换回 WGS 84
gdf = gdf.to_crs("EPSG:4326")

# 删除不需要的 sjoin 产生的索引列
gdf.drop(columns=['index_right'], inplace=True, errors='ignore')

# 保存最终结果
gdf.to_file(output_shapefile)
print(f"已成功进行空间连接，并保存到 {output_shapefile}")

已成功进行空间连接，并保存到 D:\file\e_index\223_cities_cluster_with_index.shp


In [34]:
import geopandas as gpd

# 设置输入和输出文件路径
input_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp"
output_shapefile = r"D:\file\e_index\223_cities_cluster_with_index_original.shp"
projection_file = r"D:\file\d_som\223_cities_combined_moving_ave_cluster_sort_xy.shp"

# 读取输入 Shapefile
gdf = gpd.read_file(input_shapefile)

# 读取投影 Shapefile
proj_gdf = gpd.read_file(projection_file)

utm_crs = gdf.estimate_utm_crs()
gdf = gdf.to_crs(utm_crs)
proj_gdf = proj_gdf.to_crs(utm_crs)

# 用缓冲区进行新的匹配
gdf_buffered = gdf.copy()
gdf_buffered['geometry'] = gdf_buffered.geometry.buffer(5)  # 5米缓冲区


gdf = gdf_buffered.sjoin(proj_gdf[['geometry', 'Cluster', 'Place', 'Place ID', 'Sum ID', 'City Name']], how='left', predicate='intersects')

# 5. 替换 "Sum ID" 为空的情况，并同步修改 'Cluster', 'Place', 'Place ID'
fill_columns = ['Sum ID', 'Cluster', 'Place', 'Place ID']
for col in fill_columns:
    gdf[col] = gdf[col].fillna(-9999)

# 6. 转换回 WGS 84 坐标系
gdf = gdf.to_crs("EPSG:4326")

# 7. 保存到 Shapefile
gdf.to_file(output_shapefile)

print(f"已成功进行空间连接，未匹配数据设为 -9999，并保存到 {output_shapefile}")

C:\Users\owner\AppData\Local\Temp\ipykernel_29548\3174885335.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile)


已成功进行空间连接，未匹配数据设为 -9999，并保存到 D:\file\e_index\223_cities_cluster_with_index_original.shp


In [30]:
import geopandas as gpd

# 读取 shapefile
shp_path = r"D:\file\e_index\223_cities_cluster_with_index_original.shp"
gdf = gpd.read_file(shp_path)

print("'Sum ID' 列统计情况：")
print(gdf['Sum ID'].describe())  # 统计描述
print("\n唯一值数量：", gdf['Sum ID'].nunique())  # 统计唯一值个数
print("\n前10个 Sum ID 频次：")
print(gdf['Sum ID'].value_counts().head(10))  # 查看最常见的 10 个值及其数量

'Sum ID' 列统计情况：
count    151349.000000
mean      75676.392933
std       43692.081203
min           0.000000
25%       37839.000000
50%       75676.000000
75%      113514.000000
max      151353.000000
Name: Sum ID, dtype: float64

唯一值数量： 151348

前10个 Sum ID 频次：
91495.0     2
0.0         1
143256.0    1
143258.0    1
143259.0    1
143260.0    1
143261.0    1
143262.0    1
143263.0    1
143264.0    1
143265.0    1
143266.0    1
143267.0    1
143268.0    1
143269.0    1
143270.0    1
143271.0    1
143257.0    1
143255.0    1
143273.0    1
143254.0    1
143239.0    1
143240.0    1
143241.0    1
143242.0    1
143243.0    1
143244.0    1
143245.0    1
143246.0    1
143247.0    1
143248.0    1
143249.0    1
143250.0    1
143251.0    1
143252.0    1
143253.0    1
143272.0    1
143274.0    1
143163.0    1
143275.0    1
143294.0    1
143295.0    1
143296.0    1
143297.0    1
143298.0    1
143299.0    1
143300.0    1
143301.0    1
143302.0    1
143303.0    1
Name: Sum ID, dtype: int64
'Sum ID' 列统计

In [ ]:
'''
这里需要手动删除重复值 和 nodata值
将 223_cities_cluster_with_index_original.shp
->>> 打开arcgis 把Sum ID = 91495的右边的那个栅格删了
223_cities_cluster_with_index.shp
'''

In [9]:
import geopandas as gpd

# 读取 shapefile
shp_path = r"D:\file\e_index\223_cities_cluster_with_index.shp"
gdf = gpd.read_file(shp_path)

print("'Sum ID' 列统计情况：")
print(gdf['Sum ID'].describe())  # 统计描述
print("\n唯一值数量：", gdf['Sum ID'].nunique())  # 统计唯一值个数
print("\n前10个 Sum ID 频次：")
print(gdf['Sum ID'].value_counts().head(10))  # 查看最常见的 10 个值及其数量

'Sum ID' 列统计情况：
count    151348.000000
mean      75676.288415
std       43692.206626
min           0.000000
25%       37838.750000
50%       75675.500000
75%      113514.250000
max      151353.000000
Name: Sum ID, dtype: float64

唯一值数量： 151348

前10个 Sum ID 频次：
0.0         1
143273.0    1
143257.0    1
143258.0    1
143259.0    1
143260.0    1
143261.0    1
143262.0    1
143263.0    1
143264.0    1
Name: Sum ID, dtype: int64


# 11_01 This code aims to normalize the ‘MEAN’, ‘FAR’, and ‘biodensity’ indicators.

In [5]:
# import geopandas as gpd
# from sklearn.preprocessing import MinMaxScaler
#
# # 读取 Shapefile 文件
# input_shp_path = r"D:\file\e_index\223_cities_cluster_with_index.shp"
# output_shp_path = r"D:\file\e_index\223_cities_cluster_with_index_normal.shp"
#
# # 读取数据
# gdf = gpd.read_file(input_shp_path)
#
# # 选择需要归一化的列
# columns_to_normalize = ['MEAN', 'FAR', 'Candensity','CanovRatio']
#
# # 初始化 MinMaxScaler
# scaler = MinMaxScaler()
#
# # 进行归一化处理
# gdf[columns_to_normalize] = scaler.fit_transform(gdf[columns_to_normalize])
#
# # 保存结果到新的 Shapefile 文件
# gdf.to_file(output_shp_path)
#
# print(f"归一化处理完成，结果已保存到 {output_shp_path}")


归一化处理完成，结果已保存到 D:\file\e_index\223_cities_cluster_with_index_normal.shp


In [6]:
# import pandas as pd
# import geopandas as gpd
# from sklearn.preprocessing import MinMaxScaler
#
# # 读取 Shapefile 文件
# input_shp_path = r"D:\file\e_index\223_cities_cluster_with_index.shp"
# output_shp_path = r"D:\file\e_index\223_cities_cluster_with_index_normal.shp"
#
# # 读取数据
# gdf = gpd.read_file(input_shp_path)
#
# # 选择需要归一化的列
# columns_to_normalize = ['MEAN', 'FAR', 'Candensity', 'CanovRatio']
#
# # 归一化前检查数据
# print("归一化前数据统计：")
# print(gdf[columns_to_normalize].describe())
#
# # 初始化 MinMaxScaler
# scaler = MinMaxScaler()
#
# # 进行归一化处理
# gdf[columns_to_normalize] = scaler.fit_transform(gdf[columns_to_normalize])
#
# # 归一化后检查数据
# print("\n归一化后数据统计：")
# print(gdf[columns_to_normalize].describe())
#
# # 保存结果到新的 Shapefile 文件
# gdf.to_file(output_shp_path)
#
# print(f"归一化处理完成，结果已保存到 {output_shp_path}")


归一化前数据统计：
                MEAN           FAR     Candensity     CanovRatio
count  150651.000000  1.506200e+05  150616.000000  150616.000000
mean        1.771554  4.506156e-01       0.583514       2.331681
std         1.672250  5.395749e-01       0.238900       2.454623
min         0.000148  1.956096e-08       0.000726       0.000007
25%         0.569411  7.443486e-02       0.397272       0.435813
50%         1.313960  2.610610e-01       0.608617       1.381586
75%         2.363189  6.102097e-01       0.782196       3.590397
max        16.534384  5.286934e+00       1.149691      19.507178

归一化后数据统计：
                MEAN            FAR     Candensity     CanovRatio
count  150651.000000  150620.000000  150616.000000  150616.000000
mean        0.107136       0.085232       0.507229       0.119529
std         0.101139       0.102058       0.207926       0.125832
min         0.000000       0.000000       0.000000       0.000000
25%         0.034429       0.014079       0.345133       0.02234

# 11_01 shapefile into csv

In [35]:
import geopandas as gpd
import pandas as pd
# 读取 Shapefile 文件
shp_path = r"D:\file\e_index\223_cities_cluster_with_index.shp"
output_path = r"D:\file\e_index\223_cities_cluster_with_index.csv"

# # 使用 GeoPandas 读取 Shapefile
gdf = gpd.read_file(shp_path)

# 将 GeoDataFrame 转换为 DataFrame
df = pd.DataFrame(gdf)

# 保存为 CSV 文件
df.to_csv(output_path, index=False)

print(f"Shapefile 已成功转换为 CSV，结果已保存到 {output_path}")

Shapefile 已成功转换为 CSV，结果已保存到 D:\file\e_index\223_cities_cluster_with_index.csv


In [8]:
# import geopandas as gpd
#
# # # 读取 Shapefile 文件
# # shp_path = r"D:\file\e_index\223_cities_cluster_with_index.shp"#_normal
# # output_path = r"D:\file\e_index/223_cities_cluster_with_index.csv"
# # 读取 Shapefile 文件
# shp_path = r"D:\file\e_index\223_cities_cluster_with_index_normal.shp"#_normal
# output_path = r"D:\file\e_index/223_cities_cluster_with_index_normal.csv"
# # 使用 GeoPandas 读取 Shapefile
# gdf = gpd.read_file(shp_path)
#
# # 将 GeoDataFrame 转换为 DataFrame
# df = pd.DataFrame(gdf)
#
# # 保存为 CSV 文件
# df.to_csv(output_path, index=False)
#
# print(f"Shapefile 已成功转换为 CSV，结果已保存到 {output_path}")


Shapefile 已成功转换为 CSV，结果已保存到 D:\file\e_index/223_cities_cluster_with_index_normal.csv


# 11_02 Plot the corresponding indicator images.


In [36]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
#
# def plot_cluster_histograms(input_csv_path, columns_to_plot, K):
#     # 读取数据
#     df = pd.read_csv(input_csv_path)
#
#     # 创建绘图
#     for cluster_id in range(K):
#         # 选择属于当前 cluster_id 的数据
#         cluster_data = df[df[f'Cluster'] == cluster_id]
#         # 过滤掉值为 -9999 的数据
#         cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]
#
#
#         # **输出当前 cluster 变量的统计信息**
#         print(f"\n==== Cluster {cluster_id} (K={K}) data ====")
#         print(cluster_data[columns_to_plot].describe())
#
#         # 创建直方图图像
#         plt.figure(figsize=(10, 6))
#
#         # 绘制每个列的直方图
#         for column in columns_to_plot:
#             sns.histplot(cluster_data[column], bins=10, kde=False, label=f'{column} Histogram', stat="density")
#             #print(f"绘制直方图: {column} (Cluster {cluster_id}) - 数据点数: {cluster_data[column].count()}")
#
#         # 设置图像标题和标签
#         plt.title(f'Cluster {cluster_id} Histograms (K={K})')
#         plt.xlabel('Value')
#         plt.ylabel('Density')
#
#         # # 固定横纵坐标范围
#         # plt.xlim(-0.5,20)
#         # plt.ylim(0, 5)
#
#         plt.legend()
#
#         # 保存直方图图像
#         output_hist_path = os.path.join (output_folder, rf'K={K}_histogram_cluster_{cluster_id}.png')
#         plt.savefig(output_hist_path)
#         plt.close()
#         #print(f"直方图已保存: {output_hist_path}")
#
#         # 创建 KDE 图像
#         plt.figure(figsize=(10, 6))
#
#         # 绘制每个列的 KDE 图
#         for column in columns_to_plot:
#             sns.kdeplot(cluster_data[column], label=f'{column} KDE')
#             #print(f"绘制 KDE: {column} (Cluster {cluster_id}) - 数据点数: {cluster_data[column].count()}")
#
#         # 设置图像标题和标签
#         plt.title(f'Cluster {cluster_id} KDE Plots (K={K})')
#         plt.xlabel('Value')
#         plt.ylabel('Density')
#
#         # # 固定横纵坐标范围
#         # plt.xlim(-0.5,20)
#         # plt.ylim(0, 5)
#
#         plt.legend()
#
#         # 保存 KDE 图像
#         output_kde_path = os.path.join (output_folder, rf'K={K}_kdeplot_cluster_{cluster_id}.png')
#         plt.savefig(output_kde_path)
#         plt.close()
#         #print(f"KDE 图像已保存: {output_kde_path}")
#
# def plot_cluster_analysis(input_csv_path, columns_to_plot, K):
#     # 读取数据
#     df = pd.read_csv(input_csv_path)
#
#     # 创建绘图
#     for cluster_id in range(K):
#         # 选择属于当前 cluster_id 的数据
#         cluster_data = df[df[f'Cluster'] == cluster_id]
#         # 过滤掉值为 -9999 的数据
#         cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]
#
#         # **输出统计信息**
#         #print(f"\n==== Cluster {cluster_id} (K={K}) data ====")
#         #print(cluster_data[columns_to_plot].describe())
#
#         # 将数据从宽格式转换为长格式
#         cluster_data_melted = cluster_data.melt(id_vars=[f'Cluster'], value_vars=columns_to_plot, var_name='Variable', value_name='Value')
#
#         # 创建一个新的图像
#         plt.figure(figsize=(12, 8))
#
#         # 绘制小提琴图
#         sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')
#
#         # 绘制箱线图
#         sns.boxplot(x='Variable', y='Value', data=cluster_data_melted, whis=1.5, width=0.2, palette='Set2', fliersize=0)
#
#         # **输出每个变量的统计信息**
#         # for column in columns_to_plot:
#         #     print(f"绘制箱线图 & 小提琴图: {column} (Cluster {cluster_id}) - 数据点数: {cluster_data[column].count()}")
#
#         # 设置图像标题和标签
#         plt.title(f'Cluster {cluster_id} Data Distribution')
#         plt.xlabel('Variable')
#         plt.ylabel('Value')
#         # plt.ylim(-1.5, 14)
#         # 调整布局
#         plt.tight_layout()
#
#         # 保存图像
#         output_image_path = os.path.join (output_folder, rf'K={K}_boxplot_cluster_{cluster_id}.png')
#
#         plt.savefig(output_image_path)
#         plt.close()
#         #print(f"箱线图 & 小提琴图已保存: {output_image_path}")


In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

def plot_cluster_histograms(input_csv_path, columns_to_plot, K, output_folder):
    # 读取数据
    df = pd.read_csv(input_csv_path)

    # 创建一个空的 DataFrame 用于存储所有 cluster 的统计数据
    all_cluster_stats = []

    # 创建绘图
    for cluster_id in range(K):
        # 选择属于当前 cluster_id 的数据
        cluster_data = df[df[f'Cluster'] == cluster_id]
        # 过滤掉值为 -9999 的数据
        cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]
        # 输出 describe() 的 count 为整数
        describe_output = cluster_data[columns_to_plot].describe().transpose()
        describe_output['count'] = describe_output['count'].astype(int)

        # 将统计信息添加到列表中
        describe_output['Cluster'] = cluster_id  # 标明当前的 cluster_id
        all_cluster_stats.append(describe_output)

        # 输出当前 cluster 变量的统计信息
        print(f"\n==== Cluster {cluster_id} (K={K}) data ====")
        print(describe_output)

        # 创建直方图图像
        plt.figure(figsize=(10, 6))
        for column in columns_to_plot:
            sns.histplot(cluster_data[column], bins=10, kde=False, label=f'{column} Histogram', stat="density")

        plt.title(f'Cluster {cluster_id} Histograms (K={K})')
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.xlim(-0.5, 10)
        plt.ylim(0, 5)
        plt.legend()

        # 保存直方图图像
        output_hist_path = os.path.join(output_folder, rf'K={K}_histogram_cluster_{cluster_id}.png')
        plt.savefig(output_hist_path)
        plt.close()

        # 创建 KDE 图像
        plt.figure(figsize=(10, 6))
        for column in columns_to_plot:
            sns.kdeplot(cluster_data[column], label=f'{column} KDE')

        plt.title(f'Cluster {cluster_id} KDE Plots (K={K})')
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.xlim(-0.5, 10)
        plt.ylim(0, 5)
        plt.legend()

        # 保存 KDE 图像
        output_kde_path = os.path.join(output_folder, rf'K={K}_kdeplot_cluster_{cluster_id}.png')
        plt.savefig(output_kde_path)
        plt.close()

        # 创建折线图
        plt.figure(figsize=(10, 6))
        for column in columns_to_plot:
            value_counts = cluster_data[column].value_counts().sort_index()
            plt.plot(value_counts.index, value_counts.values, label=f'{column} Lineplot')

        plt.title(f'Cluster {cluster_id} Line Plots (K={K})')
        plt.xlabel('Value')
        plt.ylabel('Count')
        plt.xlim(0, 3)
        plt.ylim(1, 3)
        plt.legend()

        # 保存折线图像
        output_line_path = os.path.join(output_folder, rf'K={K}_lineplot_cluster_{cluster_id}.png')
        plt.savefig(output_line_path)
        plt.close()

    # 合并所有统计数据到一个 DataFrame
    stats_df = pd.concat(all_cluster_stats)
    stats_df.to_csv(os.path.join(output_folder, 'cluster_stats.csv'))

def plot_cluster_analysis(input_csv_path, columns_to_plot, K, output_folder):
    # 读取数据
    df = pd.read_csv(input_csv_path)

    # 创建绘图
    for cluster_id in range(K):
        # 选择属于当前 cluster_id 的数据
        cluster_data = df[df[f'Cluster'] == cluster_id]
        # 过滤掉值为 -9999 的数据
        cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]

        # 将数据从宽格式转换为长格式
        cluster_data_melted = cluster_data.melt(id_vars=[f'Cluster'], value_vars=columns_to_plot, var_name='Variable', value_name='Value')

        # 创建一个新的图像
        plt.figure(figsize=(12, 8))

        # 绘制小提琴图
        sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')

        # 绘制箱线图
        sns.boxplot(x='Variable', y='Value', data=cluster_data_melted, whis=1.5, width=0.2, palette='Set2', fliersize=0)

        # 设置图像标题和标签
        plt.title(f'Cluster {cluster_id} Data Distribution')
        plt.xlabel('Variable')
        plt.ylabel('Value')
        plt.ylim(-1, 15)
        plt.tight_layout()

        # 保存图像
        output_image_path = os.path.join(output_folder, rf'K={K}_boxplot_cluster_{cluster_id}.png')

        plt.savefig(output_image_path)
        plt.close()

# 运行代码
input_csv_path = r"D:\file\e_index/223_cities_cluster_with_index.csv"
columns_to_plot = ['BldHeight', 'CoverRatio', 'FAR', 'GreenRatio', 'GreenVolRa']

for K in range(11, 12):
    output_folder = rf'D:\file\e_index\fig\bui'
    os.makedirs(output_folder, exist_ok=True)

    plot_cluster_histograms(input_csv_path, columns_to_plot, K, output_folder)
    plot_cluster_analysis(input_csv_path, columns_to_plot, K, output_folder)



==== Cluster 0 (K=11) data ====
            count      mean       std           min       25%       50%  \
BldHeight    4157  1.371703  1.320751  5.800000e-04  0.326603  0.971972   
CoverRatio   4157  0.527208  0.249925  1.859260e-03  0.327462  0.538953   
FAR          4157  0.328535  0.410654  3.819229e-07  0.035528  0.169883   
GreenRatio   4157  0.567927  0.292117  1.452547e-03  0.273079  0.645657   
GreenVolRa   4157  3.554311  3.319271  1.452547e-05  0.229706  3.159804   

                 75%        max  Cluster  
BldHeight   1.954704  10.281525        0  
CoverRatio  0.722787   1.005743        0  
FAR         0.459839   3.172862        0  
GreenRatio  0.827952   1.063264        0  
GreenVolRa  6.051411  15.808450        0  

==== Cluster 1 (K=11) data ====
            count      mean       std           min       25%       50%  \
BldHeight   20088  1.121718  1.253983  7.250000e-04  0.279858  0.667813   
CoverRatio  20088  0.461644  0.228708  3.486112e-04  0.289347  0.442794   


C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2715699108.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')
C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2715699108.py:105: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')
C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2715699108.py:108: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Variable', y='Value', data=cluster_data_melted, whi

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

def plot_cluster_histograms(input_csv_path, columns_to_plot, K, output_folder):
    # 读取数据
    df = pd.read_csv(input_csv_path)

    # 用来存储统计信息
    stats_df = pd.DataFrame()

    # 创建绘图
    for cluster_id in range(K):
        # 选择属于当前 cluster_id 的数据
        cluster_data = df[df[f'Cluster'] == cluster_id]
        # 过滤掉值为 -9999 的数据
        cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]
        # 假设你的列值表示距离，可以进行如下条件过滤
        cluster_data = cluster_data[(cluster_data[columns_to_plot] <= 1000).all(axis=1)]

        # 获取统计信息并添加到 DataFrame 中
        describe_output = cluster_data[columns_to_plot].describe().transpose()
        describe_output['Cluster'] = cluster_id  # 添加 Cluster 信息
        stats_df = pd.concat([stats_df, describe_output])

        # 创建直方图图像
        plt.figure(figsize=(10, 6))

        # 绘制每个列的直方图
        for column in columns_to_plot:
            sns.histplot(cluster_data[column], bins=10, kde=False, label=f'{column} Histogram', stat="density")

        # 设置图像标题和标签
        plt.title(f'Cluster {cluster_id} Histograms (K={K})')
        plt.xlabel('Value')
        plt.ylabel('Density')

        # 固定横纵坐标范围
        plt.xlim(-0.5, 1000)
        plt.ylim(0, 5)

        plt.legend()

        # 保存直方图图像
        output_hist_path = os.path.join(output_folder, rf'K={K}_histogram_cluster_{cluster_id}.png')
        plt.savefig(output_hist_path)
        plt.close()

    # 将统计信息导出为 CSV 文件
    stats_output_path = os.path.join(output_folder, rf'K={K}_cluster_stats.csv')
    stats_df.to_csv(stats_output_path)
    print(f"统计结果已保存: {stats_output_path}")

def plot_cluster_analysis(input_csv_path, columns_to_plot, K, output_folder):
    # 读取数据
    df = pd.read_csv(input_csv_path)

    # 创建绘图
    for cluster_id in range(K):
        # 选择属于当前 cluster_id 的数据
        cluster_data = df[df[f'Cluster'] == cluster_id]
        # 过滤掉值为 -9999 的数据
        cluster_data = cluster_data[~cluster_data[columns_to_plot].isin([-9999]).any(axis=1)]
        # 假设你的列值表示距离，可以进行如下条件过滤
        cluster_data = cluster_data[(cluster_data[columns_to_plot] <= 1000).all(axis=1)]

        # 将数据从宽格式转换为长格式
        cluster_data_melted = cluster_data.melt(id_vars=[f'Cluster'], value_vars=columns_to_plot, var_name='Variable', value_name='Value')

        # 创建一个新的图像
        plt.figure(figsize=(12, 8))

        # 绘制小提琴图
        sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')

        # 绘制箱线图
        sns.boxplot(x='Variable', y='Value', data=cluster_data_melted, whis=1.5, width=0.2, palette='Set2', fliersize=0)

        # 设置图像标题和标签
        plt.title(f'Cluster {cluster_id} Data Distribution')
        plt.xlabel('Variable')
        plt.ylabel('Value')
        # 调整布局
        plt.tight_layout()
        plt.ylim(-200, 1200)

        # 保存图像
        output_image_path = os.path.join(output_folder, rf'K={K}_boxplot_cluster_{cluster_id}.png')

        plt.savefig(output_image_path)
        plt.close()

# 运行代码
input_csv_path = r"D:\file\e_index/223_cities_cluster_with_index.csv"
columns_to_plot = ['MIN_DistWB', 'MIN_DistGL', 'MIN_DistMT']

for K in range(11, 12):
    output_folder = rf'D:\file\e_index\fig\dist'
    os.makedirs(output_folder, exist_ok=True)

    plot_cluster_histograms(input_csv_path, columns_to_plot, K, output_folder)
    plot_cluster_analysis(input_csv_path, columns_to_plot, K, output_folder)


统计结果已保存: D:\file\e_index\fig\dist\K=11_cluster_stats.csv


C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2978644814.py:75: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')
C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2978644814.py:75: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(x='Variable', y='Value', data=cluster_data_melted, inner=None, scale='width', palette='Set3')
C:\Users\owner\AppData\Local\Temp\ipykernel_29548\2978644814.py:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Variable', y='Value', data=cluster_data_melted, whis=1